# 通过微调进行迁移学习
前面我们介绍了如何训练卷积神经网络进行图像分类，可能你已经注意到了，训练一个卷积网络是特别耗费时间的，特别是一个比较深的卷积网络，而且可能因为训练方法不当导致训练不收敛的问题，就算训练好了网络，还有可能出现过拟合的问题，所以由此可见能够得到一个好的模型有多么困难。

有的时候，我们的数据集还特别少，这对于我们来讲无异于雪上加霜，因为少的数据集意味着非常高的风险过拟合，那么我们有没有办法在某种程度上避免这个问题呢？其实现在有一种方法特别流行，大家一直在使用，那就是微调(fine-tuning)，在介绍微调之前，我们先介绍一个数据集 ImageNet。

## ImageNet
ImageNet 是一个计算机视觉系统识别项目，是目前世界上最大的图像识别数据库，由斯坦福大学组织建立，大约有 1500 万张图片，2.2 万中类别，其中 ISLVRC 作为其子集是学术界中使用最为广泛的公开数据集，一共有 1281167 张图片作为训练集，50000 张图片作为验证集，一共是 1000 分类，是目前测试网络性能的标杆。

我们说的这个数据集有什么用呢？我们又不关心这个数据集，但是对于我们自己的问题，我们有没有办法借助 ImageNet 中的数据集来提升模型效果，比如我们要做一个猫狗分类器，但是我们现在只有几百张图片，肯定不够，ImageNet 中有很多关于猫狗的图片，我们如果能够把这些图片拿过来训练，不就能够提升模型性能了吗？

但是这种做法太麻烦了，从 ImageNet 中寻找这些图片就很困难，如果做另外一个问题又要去找新的图片，所以直接找图片并不靠谱，那么有没有办法能够让我们不去找这些图片，又能使用这些图片呢？

非常简单，我们可以使用在 ImageNet 上训练好的网路，然后把这个网络在放到我们自己的数据集上进行训练不就好了。这个方法就叫做微调，这十分形象，相当于把一个已经很厉害的模型再微调到我们自己的数据集上来，也可称为迁移学习。

迁移学习的方法非常简单，将预训练的模型导入，然后将最后的分类全连接层换成适合我们自己问题的全连接层，然后开始训练，可以固定卷积层的参数，也可以不固定进行训练，最后能够非常有效的得到结果

pytorch 一直为我们内置了前面我们讲过的那些著名网络的预训练模型，不需要我们自己去 ImageNet 上训练了，模型都在 `torchvision.models` 里面，比如我们想使用预训练的 50 层 resnet，就可以用 `torchvision.models.resnet50(pretrained=True)` 来得到

下面我们用一个例子来演示一些微调

In [ ]:
import sys
sys.path.append('..')

import numpy as np

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import models
from torchvision import transforms as tfs
from torchvision.datasets import ImageFolder

首先我们点击下面的[链接](https://download.pytorch.org/tutorial/hymenoptera_data.zip)获得数据集，终端可以使用

`wget https://download.pytorch.org/tutorial/hymenoptera_data.zip`

下载完成之后，我们将其解压放在程序的目录下，这是一个二分类问题，区分蚂蚁和蜜蜂

我们可以可视化一下图片，看看你能不能区分出他们来

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
root_path = '/home/yang/dataset/imagenet/fruits/train/'
im_list = [os.path.join(root_path, 'kiwi', i) for i in os.listdir(root_path + 'kiwi')[:4]]
im_list += [os.path.join(root_path, 'watermelon', i) for i in os.listdir(root_path + 'watermelon')[:5]]

nrows = 5
ncols = 5
figsize = (8, 8)
_, figs = plt.subplots(nrows, ncols, figsize=figsize)
for i in range(nrows):
    for j in range(ncols):
        figs[i][j].imshow(Image.open(im_list[nrows*i+j]))
        figs[i][j].axes.get_xaxis().set_visible(False)
        figs[i][j].axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# 定义数据预处理
train_tf = tfs.Compose([
    tfs.RandomResizedCrop(224),
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),
    tfs.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 使用 ImageNet 的均值和方差
])

valid_tf = tfs.Compose([
    tfs.Resize(256),
    tfs.CenterCrop(224),
    tfs.ToTensor(),
    tfs.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# 使用 ImageFolder 定义数据集
train_set = ImageFolder('/home/yang/dataset/imagenet/fruits/train/', train_tf)
valid_set = ImageFolder('/home/yang/dataset/imagenet/fruits/val/', valid_tf)
# 使用 DataLoader 定义迭代器
train_data = DataLoader(train_set, 64, True, num_workers=4)
valid_data = DataLoader(valid_set, 8, False, num_workers=4)

In [ ]:
class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()

        # Normal convolution block followed by Batchnorm (CONV_3x3-->BN-->Relu)
        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        # Depthwise convolution block (CONV_BLK_3x3-->BN-->Relu-->CONV_1x1-->BN-->Relu)
        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [ ]:
model = MobileNet()
#print(model)
model = torch.nn.DataParallel(model).cuda()

In [ ]:
params = torch.load('model_best.pth.tar')['state_dict']

In [ ]:
model.load_state_dict(params)

In [ ]:
print(model)

In [ ]:
# 使用预训练的模型
#net = models.resnet50(pretrained=True)
#print(net)

In [ ]:
# 打出第一层的权重
#print(model.conv1.weight)

In [ ]:
# 将最后的全连接层改成二分类
model.fc = nn.Linear(2048, 2)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-4)

In [ ]:
from utils import train
train(model, train_data, valid_data, 10, optimizer, criterion)

下面我们来可视化预测的结果

In [ ]:
model = model.eval() # 将网络改为预测模式

读一张蚂蚁的图片

In [ ]:
im1 = Image.open('/home/yang/dataset/imagenet/fruits/val/watermelon/938783625.jpg')
im1

In [ ]:
im = valid_tf(im1) # 做数据预处理
out = model(Variable(im.unsqueeze(0)).cuda())
pred_label = out.max(1)[1].data[0]
print('predict label: {}'.format(train_set.classes[pred_label]))

可以看到预测的结果是对的

**小练习：看看上面的网络预测过程，多尝试几张图片进行预测**

In [ ]:
# 保持前面的卷积层参数不变
net = models.resnet50(pretrained=True)
for param in net.parameters():
    param.requires_grad = False # 将模型的参数设置为不求梯度
net.fc = nn.Linear(2048, 2)

optimizer = torch.optim.SGD(net.fc.parameters(), lr=1e-2, weight_decay=1e-4)

In [ ]:
train(net, train_data, valid_data, 20, optimizer, criterion)

可以看到只训练验证集的准确率也可以达到比较高，但是 loss 跳动比较大，因为更新的参数太少了，只有全连接层的参数

In [ ]:
# 不使用预训练的模型
net = models.resnet50()
net.fc = nn.Linear(2048, 2)

optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, weight_decay=1e-4)

In [ ]:
# 打出第一层的权重
print(net.conv1.weight)

In [ ]:
train(net, train_data, valid_data, 20, optimizer, criterion)

通过上面的结果可以看到，使用预训练的模型能够非常快的达到 95% 左右的验证集准确率，而不使用预训练模型只能到 70% 左右的验证集准确率，所以使用一个预训练的模型能够在较小的数据集上也取得一个非常好的效果，因为对于图片识别分类任务，最底层的卷积层识别的都是一些通用的特征，比如形状、纹理等等，所以对于很多图像分类、识别任务，都可以使用预训练的网络得到更好的结果。